# Домашние задание №3

In [1]:
import pandas as pd
from pprint import pprint
from copy import deepcopy

from tqdm.auto import tqdm
import pickle

from rectools import Columns
from rectools.models.popular import PopularModel
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset, Interactions
from rectools.metrics.classification import Precision, Recall
from rectools.metrics.ranking import MAP
from rectools.metrics.novelty import MeanInvUserFreq
from rectools.metrics.serendipity import Serendipity
from rectools.metrics import calc_metrics
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender

from userknn import UserKnn

In [2]:
DATA_DIR_PATH = '/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/RecomendationsService/DATA'

interactions_df = pd.read_csv(f'{DATA_DIR_PATH}/interactions.csv')

interactions_df = interactions_df.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'watched_pct': Columns.Weight
    })
interactions_df = interactions_df.drop(columns=['total_dur'])

print(interactions_df.shape)
interactions_df.head()

(5476251, 4)


,user_id,item_id,datetime,weight
0,176549,9506,2021-05-11,72.0
1,699317,1659,2021-05-29,100.0
2,656683,7107,2021-05-09,0.0
3,864613,7638,2021-07-05,100.0
4,964868,9506,2021-04-30,100.0


In [3]:
users_df = pd.read_csv(f'{DATA_DIR_PATH}/items.csv')

print(users_df.shape)
users_df.head()

(15963, 14)


,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [4]:
data_users = pd.read_csv(f'{DATA_DIR_PATH}/users.csv')

print(data_users.shape)
data_users.head()

(840197, 5)


,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [5]:
# ! если хотите быстро прогнать этот ноутбук - раскомментируйте эту строку - она уменьшает данные
# interactions = Interactions(interactions_df.sample(frac=0.35))

interactions = Interactions(interactions_df)
interactions.df.head()

,user_id,item_id,datetime,weight
0,176549,9506,2021-05-11,72.0
1,699317,1659,2021-05-29,100.0
2,656683,7107,2021-05-09,0.0
3,864613,7638,2021-07-05,100.0
4,964868,9506,2021-04-30,100.0


In [10]:
# оставляем только горячих юзеров, для холодных у нас есть популярное
splitter = TimeRangeSplitter(
    '7D',
    n_splits=5,
    filter_cold_users=True,
    filter_cold_items=True,
    filter_already_seen=True
)

Прогоним кроссвалидацию на несколкьих моделях что бы посмотреть, что даст лучший результат. Также попробуем разные количество похожих юзеров

In [12]:
models = {
    'tfidf': TFIDFRecommender,
    'cosine': CosineRecommender,
    'bm25': BM25Recommender
}
# попробуем разное количество похожих юзеров
n_users = {
    'k25': 25,
    'k30': 30,
    'k35': 35
}

# будем расчитывать следующие метрик
metrics = {
    'precision@10': Precision(k=10),
    'recall@10': Recall(k=10),
    'map@10': MAP(k=10),
    'novelty@10': MeanInvUserFreq(k=10),
    'serendipity@10': Serendipity(k=10)
}

In [ ]:
# проведем кросс валидацию
# и будем записывать логи в файлик, что бы не потерять прогресс (это очень долго)
results = []

fold_iterator = splitter.split(interactions, collect_fold_stats=True)

for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=4):
    print(f'\n==================== Fold {fold_info['i_split']}')
    pprint(fold_info)
    
    df_train = interactions.df.iloc[train_ids]
    df_test = interactions.df.iloc[test_ids][Columns.UserItem]

    catalog = df_train[Columns.Item].unique()

    for model_name, model in models.items():
        for n_users_flag, n_user in n_users.items():
            print(f'\n==== Recommender: {model_name}, K_sim_users: {n_user}')
            userknn_model = UserKnn(model(n_user), N_users=10)
            userknn_model = deepcopy(userknn_model)
            userknn_model.fit(df_train)
            recos = userknn_model.predict(df_test)
            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )
            metric_values['n_user'] = n_user
            metric_values['fold'] = fold_info['i_split']
            metric_values['model'] = model_name

            results.append(metric_values)

    with open('cv_results.pkl', 'wb') as dumpfile:
        pickle.dump(results, 'cv_results.pkl')


In [17]:
with open('cv_results.pkl', 'rb') as dumpfile:
    results = pickle.load(dumpfile)
results

[{'fold': 0,
  'model': 'cosine',
  'precision@10': 0.004457907491307554,
  'recall@10': 0.022002969952018683,
  'map@10': 0.004357348153557538,
  'novelty@10': 6.856974016422273,
  'serendipity@10': 4.633714137877612e-05,
  'n_user': 25},
 {'fold': 0,
  'model': 'cosine',
  'precision@10': 0.004467390159098093,
  'recall@10': 0.022078562263548457,
  'map@10': 0.004376041394011916,
  'novelty@10': 6.8471810728074045,
  'serendipity@10': 4.612028628980783e-05,
  'n_user': 30},
 {'fold': 0,
  'model': 'cosine',
  'precision@10': 0.004446317564008008,
  'recall@10': 0.021871648501897857,
  'map@10': 0.004307191306081466,
  'novelty@10': 6.8216151999310375,
  'serendipity@10': 4.661225939478569e-05,
  'n_user': 35},
 {'fold': 0,
  'model': 'tfidf',
  'precision@10': 0.006396586239595406,
  'recall@10': 0.032372549486836356,
  'map@10': 0.006268968471802279,
  'novelty@10': 7.32946501498472,
  'serendipity@10': 5.009519302597978e-05,
  'n_user': 25},
 {'fold': 0,
  'model': 'tfidf',
  'prec

In [16]:
cv_results_df = pd.DataFrame(results)
cv_results_df = cv_results_df.drop(columns=['fold']).groupby(['model', 'n_user']).agg(['mean'])

subset = [(metric, agg) for metric, agg in cv_results_df.columns if agg == 'mean']
colored_df =cv_results_df.style \
    .highlight_min(subset=subset, color='lightcoral', axis=0) \
    .highlight_max(subset=subset, color='lightgreen', axis=0)
colored_df

Исходя из кроссвалидации можно сделать вывод что:
1) делать большое количество юзеров бесполезно
2) tf_idf в нашем случае точнее

С холодными юзерами будет работать популярное

Обучим модель выдавать популярное. Используем это для холодных юзеров, или если у нас будет слишком мало айтемов при рекомендации горячеему пользователю.

Для экономии времени (модель у нас работает в режиме онлайн) сохраним результаты и просто будем хранить их в модели в виде списка

In [22]:
mod = PopularModel()
ds = Dataset.construct(interactions_df)
mod.fit(ds)

In [26]:
popular_items_list = mod.recommend([1], ds, k=10, filter_viewed=False)['item_id'].to_list()
popular_items_list

[10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142, 6809]

In [ ]:
with open('popular_items_list.pkl', 'wb') as pop_list_file: 
    pickle.dump(popular_items_list, pop_list_file)

Теперь обучим UserKNN на всем датасете

In [7]:
userknn_model = UserKnn(model=TFIDFRecommender(K=25), popular=popular_items_list, N_users=20)
userknn_model.fit(interactions_df)
with open('user_knn.pkl', 'wb') as userknn_model_file:
    pickle.dump(userknn_model, userknn_model_file)

/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/RecomendationsService/.venv/lib/python3.10/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.028964757919311523 seconds
  warnings.warn(


  0%|          | 0/395111 [00:00<?, ?it/s]

In [10]:
userknn_model.predict_single(user_id=2, N_recs=10) # работает

[383, 10636, 7106, 8482, 561, 15196, 6220, 5411, 4436, 10440]